In [51]:
import os
import torch
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from torch import nn, optim
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.transforms.functional import to_pil_image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [12]:

disease_train = load_dataset('gOLIVES/OLIVES_Dataset',
                      'disease_classification',
                      split='train',
                     )

print('finish loading disease dataset')

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

train-00000-of-00032.parquet:   0%|          | 0.00/446M [00:00<?, ?B/s]

train-00001-of-00032.parquet:   0%|          | 0.00/513M [00:00<?, ?B/s]

train-00002-of-00032.parquet:   0%|          | 0.00/489M [00:00<?, ?B/s]

...

train-00030-of-00032.parquet:   0%|          | 0.00/432M [00:00<?, ?B/s]

train-00031-of-00032.parquet:   0%|          | 0.00/488M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78822 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

finish loading disease dataset


In [13]:
bio_test = load_dataset('gOLIVES/OLIVES_Dataset',
                      'biomarker_detection',
                      split='test',
                     )

print('finish loading bio_test')

bio_train = load_dataset('gOLIVES/OLIVES_Dataset',
                      'biomarker_detection',
                      split='train',
                     )

print('finish loading bio_train')

bio_test = bio_test.with_format("torch")
bio_train = bio_train.with_format("torch")

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

train-00005-of-00032.parquet:   0%|          | 0.00/491M [00:00<?, ?B/s]

train-00006-of-00032.parquet:   0%|          | 0.00/505M [00:00<?, ?B/s]

train-00007-of-00032.parquet:   0%|          | 0.00/476M [00:00<?, ?B/s]

...

finish loading bio_test


Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

finish loading bio_train


In [15]:
class EyeDataset(Dataset):
    def __init__(self, data, augmentation=None):
        self.data = data
        self.augmentation = augmentation

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        record = self.data[index]

        # Image processing
        img = to_pil_image(record['Image'].float())
        img = img.convert('L')  # Convert to grayscale
        if self.augmentation:
            img = self.augmentation(img)

        # Clinical information processing
        clinical_features = torch.tensor([
            record['BCVA'], record['CST'], record['Eye_ID'], record['Patient_ID'],
            record['B1'], record['B2'], record['B3'], record['B4'], record['B5'], record['B6']
        ], dtype=torch.float32)
        clinical_features = torch.nan_to_num(clinical_features, nan=0.0, posinf=0.0, neginf=0.0)

        # Feature normalization
        feature_mean = clinical_features.mean()
        feature_std = clinical_features.std() + 1e-8
        clinical_features = (clinical_features - feature_mean) / feature_std

        # Binary label conversion
        labels_binary = (clinical_features > 0.5).float()

        return img, clinical_features, labels_binary


# Image augmentation setup
image_augmentation = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])



In [22]:
def balance_data(features, targets):
    sampler = RandomOverSampler(sampling_strategy='auto')
    features_balanced, targets_balanced = sampler.fit_resample(features, targets)
    return features_balanced, targets_balanced


def compute_positive_weights(targets):
    target_counts = torch.tensor([sum(targets[:, i]) for i in range(targets.shape[1])])
    positive_weights = (targets.shape[0] - target_counts) / target_counts
    return positive_weights

In [23]:
def load_olives_data(augmentation):
    bio_train_data = load_dataset('gOLIVES/OLIVES_Dataset', 'biomarker_detection',
                                  split='train')
    bio_train_data = bio_train_data.with_format("torch")

    training_dataset = EyeDataset(bio_train_data, augmentation=augmentation)
    training_loader = DataLoader(training_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
    return training_loader

train_loader = load_olives_data(image_augmentation)

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

In [40]:
def load_test_data(augmentation):
    bio_test_data = load_dataset('gOLIVES/OLIVES_Dataset', 'biomarker_detection',
                                 split='test')
    bio_test_data = bio_test_data.with_format("torch")

    test_dataset = EyeDataset(bio_test_data, augmentation=augmentation)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
    return test_loader


test_loader = load_test_data(image_augmentation)

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

In [32]:
class HybridEyeModel(nn.Module):
    def __init__(self):
        super(HybridEyeModel, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 128)

        self.clinical_fc = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        self.combined_fc = nn.Sequential(
            nn.Linear(128 + 32, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 10)  # 10 outputs for 10 labels
        )

    def forward(self, image, clinical_data):
        image_features = self.resnet(image)
        clinical_features = self.clinical_fc(clinical_data)
        combined_features = torch.cat((image_features, clinical_features), dim=1)
        output = self.combined_fc(combined_features)
        return output


In [33]:
training_metrics = {
    'accuracies': {
        'total': [],
        'B1': [], 'B2': [], 'B3': [], 'B4': [], 'B5': [], 'B6': []
    },
    'f1_scores': {
        'total': [],
        'B1': [], 'B2': [], 'B3': [], 'B4': [], 'B5': [], 'B6': []
    }
}

In [34]:
def calculate_metrics(predictions, targets):
    pred_labels = (torch.sigmoid(predictions) > 0.5).float()
    accuracies = []
    f1_scores = []

    for i in range(targets.shape[1]):
        acc = (pred_labels[:, i] == targets[:, i]).float().mean().item()
        true_positives = (pred_labels[:, i] * targets[:, i]).sum().item()
        pred_positives = pred_labels[:, i].sum().item()
        real_positives = targets[:, i].sum().item()

        precision = true_positives / (pred_positives + 1e-10)
        recall = true_positives / (real_positives + 1e-10)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-10)

        accuracies.append(acc)
        f1_scores.append(f1)

    total_acc = sum(accuracies) / len(accuracies)
    total_f1 = sum(f1_scores) / len(f1_scores)

    return total_acc, accuracies, total_f1, f1_scores

In [35]:
def plot_training_metrics(metrics, current_epoch):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    epochs = range(len(metrics['accuracies']['total']))

    ax1.plot(epochs, metrics['accuracies']['total'],
             linewidth=2, label='Total Acc')
    for i in range(1, 7):
        ax1.plot(epochs, metrics['accuracies'][f'B{i}'],
                linestyle='--', label=f'B{i} Acc')

    ax1.set_ylim(0.3, 1.0)
    ax1.grid(True)
    ax1.legend()
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.set_title('Train VS Test Loss per Epoch')

    ax2.plot(epochs, metrics['f1_scores']['total'],
             linewidth=2, label='Total F1 Score')
    for i in range(1, 7):
        ax2.plot(epochs, metrics['f1_scores'][f'B{i}'],
                linestyle='--', label=f'B{i} F1 Score')

    ax2.set_ylim(0, 0.8)
    ax2.grid(True)
    ax2.legend()
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('F1 Score')

    plt.tight_layout()
    plt.savefig(f'training_metrics_epoch_{current_epoch}.png', dpi=300, bbox_inches='tight')
    plt.close()


In [38]:
def train_eye_model(model, loader, loss_fn, optimizer, grad_scaler, epochs=10):
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        epoch_outputs = []
        epoch_labels = []

        progress_bar = tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

        for images, clinical_data, labels in progress_bar:
            images = images.to(device)
            clinical_data = clinical_data.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                outputs = model(images, clinical_data)
                loss = loss_fn(outputs, labels)

            grad_scaler.scale(loss).backward()
            grad_scaler.step(optimizer)
            grad_scaler.update()

            total_loss += loss.item()

            # Store batch outputs and labels for metric calculation
            epoch_outputs.append(outputs.detach().cpu())  # Move to CPU
            epoch_labels.append(labels.detach().cpu())    # Move to CPU

            progress_bar.set_postfix(loss=loss.item())

        # Calculate metrics for the epoch
        epoch_outputs = torch.cat(epoch_outputs)
        epoch_labels = torch.cat(epoch_labels)
        total_acc, accuracies, total_f1, f1_scores = calculate_metrics(epoch_outputs, epoch_labels)

        # Store metrics
        training_metrics['accuracies']['total'].append(total_acc)
        training_metrics['f1_scores']['total'].append(total_f1)
        for i in range(6):
            training_metrics['accuracies'][f'B{i+1}'].append(accuracies[i])
            training_metrics['f1_scores'][f'B{i+1}'].append(f1_scores[i])

        print(f"Epoch {epoch + 1}, Avg Loss: {total_loss / len(loader):.4f}, "
              f"Accuracy: {total_acc:.4f}, F1 Score: {total_f1:.4f}")

        # Plot metrics every N epochs or at the end
        if (epoch + 1) % 10 == 0 or epoch == epochs - 1:
            plot_training_metrics(training_metrics, epoch + 1)

    return training_metrics

In [39]:
# Set up device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridEyeModel().to(device)

# Set up loss function and optimizer
pos_weight = torch.ones(10).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler = torch.cuda.amp.GradScaler()

# Train the model
train_eye_model(model, train_loader, criterion, optimizer, scaler)

# Save the model
torch.save(model.state_dict(), "hybrid_eye_model.pth")


<ipython-input-39-34d8bd3f89bd>:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/10:   0%|          | 0/4927 [00:00<?, ?it/s]<ipython-input-38-ebe1d2f36e23>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1, Avg Loss: 0.0477, Accuracy: 0.9827, F1 Score: 0.2950


Epoch 2, Avg Loss: 0.0172, Accuracy: 0.9935, F1 Score: 0.3489


Epoch 3, Avg Loss: 0.0135, Accuracy: 0.9949, F1 Score: 0.3611


Epoch 4, Avg Loss: 0.0113, Accuracy: 0.9957, F1 Score: 0.3677


...

Epoch 6, Avg Loss: 0.0077, Accuracy: 0.9971, F1 Score: 0.3771


Epoch 7, Avg Loss: 0.0063, Accuracy: 0.9976, F1 Score: 0.3799


Epoch 8, Avg Loss: 0.0053, Accuracy: 0.9980, F1 Score: 0.3834


Epoch 9, Avg Loss: 0.0046, Accuracy: 0.9983, F1 Score: 0.3861


Epoch 10, Avg Loss: 0.0042, Accuracy: 0.9985, F1 Score: 0.3869


In [56]:
def evaluate_eye_model(model, loader, loss_fn):
    model.eval()
    total_loss = 0
    epoch_outputs = []
    epoch_labels = []

    with torch.no_grad():
        progress_bar = tqdm(loader, desc="Evaluating", leave=False)
        for images, clinical_data, labels in progress_bar:
            images = images.to(device)
            clinical_data = clinical_data.to(device)
            labels = labels.to(device)

            outputs = model(images, clinical_data)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            # Store batch outputs and labels for metric calculation
            epoch_outputs.append(outputs.cpu())
            epoch_labels.append(labels.cpu())

            progress_bar.set_postfix(loss=loss.item())

    # Calculate metrics
    epoch_outputs = torch.cat(epoch_outputs)
    epoch_labels = torch.cat(epoch_labels)
    total_acc, accuracies, total_f1, f1_scores = calculate_metrics(epoch_outputs, epoch_labels)

    # Classification report
    pred_labels = (torch.sigmoid(epoch_outputs) > 0.5).int().numpy()
    true_labels = epoch_labels.int().numpy()
    report = classification_report(true_labels, pred_labels, target_names=[f'B{i+1}' for i in range(10)], zero_division=0)


    print(f"Evaluation Results - Avg Loss: {total_loss / len(loader):.4f}, "
          f"Accuracy: {total_acc:.4f}, F1 Score: {total_f1:.4f}\n")
    print(report)

    return total_acc, total_f1

In [57]:
evaluate_eye_model(model, test_loader, criterion)

Evaluating:   0%|          | 0/242 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
                                                                          

Evaluation Results - Avg Loss: 0.0919, Accuracy: 0.9779, F1 Score: 0.1799

              precision    recall  f1-score   support

          B1       1.00      0.67      0.80      2401
          B2       1.00      1.00      1.00      3871
          B3       0.00      0.00      0.00         0
          B4       0.00      0.00      0.00         0
          B5       0.00      0.00      0.00         0
          B6       0.00      0.00      0.00         0
          B7       0.00      0.00      0.00         0
          B8       0.00      0.00      0.00         0
          B9       0.00      0.00      0.00         0
         B10       0.00      0.00      0.00         0

   micro avg       0.99      0.87      0.93      6272
   macro avg       0.20      0.17      0.18      6272
weighted avg       1.00      0.87      0.92      6272
 samples avg       0.99      0.90      0.93      6272



(0.9778868496418, 0.17986006995521073)